In [1]:
from api_key import get_api_key
import google.generativeai as genai
from vpn_control import windscribe
import time
import re
import random
import datetime
import pandas as pd
import matplotlib
import numpy as np
import seaborn 
import requests


In [2]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [3]:
passw = "2002"

In [4]:
def setup_prompt_env(passw, extra_instruct):


  genai.configure(api_key=get_api_key(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 300,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = f"""You must modify the log template such that it coveys the same infromation but is structred differently/uses different words. You can add or remove <*> wildcard which will contain variables later. Only use sensible replacements in the context.  Additonally {extra_instruct}. Finally you will be given a change_level token at the start of the prompt which tells you how much of the original log should be modified from 0 to 10, where 0 is no change and 10 is every token/word and order can be changed. Pad the log with a begin and end token <B> <E>, dont give addtional text. Find am example below
  
  ORGINAL: Node <*> experienced data loss at <*>
  change_level 0: Node <*> experienced data loss at <*>
  change_level 5: <*> node data loss at <*>  
  change_level 10: HDFS node number <*> lost data due to unkown falure at time <*> 
  
  Dont give extra text and do it for all inputs below"""
  

  model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [5]:
def augment_template_LLM(message, convo, change_Level, extra_instruct):
    message =  str(change_Level) + "\n" + message 
    convo.send_message(message)
    suggestion = convo.last.text
    print(suggestion)
    return suggestion

In [6]:
def setup_LLM_aug(passw, extra_instruct, message, change_level):
    convo_create = setup_prompt_env(passw, extra_instruct)
    augment_template_LLM(message, convo_create, change_level, extra_instruct)

In [7]:
setup_LLM_aug(passw, "make it sound like it was written by a pirate", "Computer <*> overheated and HDD failed at <*>", 10)

<B> Shiver me timbers!  Drive on node <*> walked the plank at <*>! <E> 



In [8]:
setup_LLM_aug(passw, "make it very robotic in language", "Computer <*> overheated and HDD failed at <*>", 10)

<B> Unit <*> suffered storage malfunction at time <*> <E> 



In [9]:
time.sleep(30)

In [10]:
setup_LLM_aug(passw, "make it very verbose", "Computer <*> overheated and HDD failed at <*>", 10)

<B> At time <*> the hard disk drive on device <*> ceased to function due to a thermal event. <E> 



In [19]:
setup_LLM_aug(passw, ".", "Computer <*> overheated and HDD failed at <*>", 0)

<B> Computer <*> overheated and HDD failed at <*> <E> 



In [20]:
setup_LLM_aug(passw, ".", "Computer <*> overheated and HDD failed at <*>", 5)

<B> <*> HDD failure due to overheat at <*> <E> 



In [21]:
setup_LLM_aug(passw, ".", "Computer <*> overheated and HDD failed at <*>", 10)

<B> <*> hard drive failed due to high temperature at time <*> <E> 



In [22]:
windscribe("disconnect")

In [23]:
asD

NameError: name 'asD' is not defined

In [ ]:
def generate_random_values_LLM(aug_template):

    placeholders = re.findall(r"<\*>", template)
    
    for placeholder in placeholders:
        prompt = f"Generate a contextually appropriate value for the placeholder in this context: {template}"
        value = get_gemini_suggestion(aug_template)
        
        template = template.replace("<*>", value, 1)
    
    return template


In [ ]:
windscribe("disconnect")